In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# imports

from google.colab import files
from os import listdir
from os.path import isfile, join
import re
from collections import Counter
from PIL import Image
import numpy as np
from keras.models import Model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from matplotlib.image import imread
from random import shuffle
from math import ceil

In [14]:
# variaveis

img_to_load = 300 # imagens a serem carregadas por vez (carregando as 10000 de uma vez sobrecarrega o sistema)
crop_size = 150 # altura e largura da imagem a ser enviada para o modelo
train = 0.7 # porcentagem de dados a ser usado para treino

In [3]:
#função para pegar só o centro da imagem

def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [4]:
#carregar nome de todas as imagens

ER_path = '/gdrive/My Drive/IDAO/train/ER/'
ER_files = [f'/gdrive/My Drive/IDAO/train/ER/{f}' for f in listdir(ER_path) if isfile(join(ER_path, f))]

NR_path = '/gdrive/My Drive/IDAO/train/NR/'
NR_files = [f'/gdrive/My Drive/IDAO/train/NR/{f}' for f in listdir(NR_path) if isfile(join(NR_path, f))]

all_files = ER_files + NR_files
shuffle(all_files)

In [5]:
# criar um dicionário para armazenar todos os dados separadamente
# isso é necessário para garantir que 70% de cada uma das classes serão usadas para treino

images = {'NR_1_keV': [], 'NR_6_keV': [], 'NR_20_keV': [], 'ER_3_keV': [], 'ER_10_keV': [], 'ER_30_keV': []}

In [6]:
# coloca todas as strings das imagens no dicionário

for particle_type in images.keys():
  images[particle_type] = [img for img in all_files if particle_type in img]

for x in images.keys():
    images[x] = images[x]
    print(f"{x}: {len(images[x])}")

NR_1_keV: 2177
NR_6_keV: 2255
NR_20_keV: 2208
ER_3_keV: 2243
ER_10_keV: 2272
ER_30_keV: 2237


In [7]:
# carrega 70% dos dados no X_train e 30% no X_test

X_train = []
X_test = []

for particle_type in images.keys():
    X_train = X_train + images[particle_type][:int(train*len(images[particle_type]))]
    X_test = X_test + images[particle_type][int(train*len(images[particle_type])):]
shuffle(X_train)
shuffle(X_test)

In [8]:
# cria o Y_train e Y_test com base no X_train e X_test criado anteriormente

Y_train = []
Y_test = []

for x in X_train:
    Y_train.append(1 if '_NR_' in x else 0)

for x in X_test:
    Y_test.append(1 if '_NR_' in x else 0)

In [ ]:
# first model test

test_model = Sequential()

test_model.add(Conv2D(32, (3, 3), input_shape=(crop_size, crop_size, 1)))
test_model.add(Activation('relu'))
test_model.add(MaxPooling2D(pool_size=(2, 2)))

test_model.add(Conv2D(32, (3, 3)))
test_model.add(Activation('relu'))
test_model.add(MaxPooling2D(pool_size=(2, 2)))

test_model.add(Conv2D(64, (3, 3)))
test_model.add(Activation('relu'))
test_model.add(MaxPooling2D(pool_size=(2, 2)))

test_model.add(Flatten())
test_model.add(Dense(64))
test_model.add(Activation('relu'))
#test_model.add(Dropout(0.5)) # treinar dpois com o dropout
test_model.add(Dense(1))
test_model.add(Activation('sigmoid'))

test_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# treina o modelo usando batches de 300 imagens por vez

for i in range(ceil(len(X_train)/img_to_load)):
#for i in range(ceil(1500/img_to_load)):
    begin = i*img_to_load
    limit = min(i*img_to_load + img_to_load, len(X_train))
    if begin == limit:
        break

    X_train_img = [crop_center(imread(file), crop_size, crop_size) for file in X_train[begin:limit]]
    Y_train_img = Y_train[begin:limit]
    print(f'loaded {begin} to {limit}')

    X_train_img = np.stack(X_train_img, axis=0)
    X_train_img = X_train_img.reshape((X_train_img.shape + (1,)))
    Y_train_img = np.stack(Y_train_img, axis=0).reshape((len(Y_train_img), 1))
    #print(X_train_img.shape)
    test_model.fit(X_train_img, Y_train_img, epochs=10, batch_size=50)

loaded 0 to 300
Epoch 1/10
6/6 [==============================] - 1s 33ms/step - loss: 1.1239 - accuracy: 0.5170
Epoch 2/10
6/6 [==============================] - 0s 24ms/step - loss: 0.6963 - accuracy: 0.4899
Epoch 3/10
6/6 [==============================] - 0s 24ms/step - loss: 0.6943 - accuracy: 0.4880
Epoch 4/10
6/6 [==============================] - 0s 23ms/step - loss: 0.6938 - accuracy: 0.5189
Epoch 5/10
6/6 [==============================] - 0s 23ms/step - loss: 0.6972 - accuracy: 0.4862
Epoch 6/10
6/6 [==============================] - 0s 25ms/step - loss: 0.6956 - accuracy: 0.4667
Epoch 7/10
6/6 [==============================] - 0s 24ms/step - loss: 0.6931 - accuracy: 0.5014
Epoch 8/10
6/6 [==============================] - 0s 24ms/step - loss: 0.6935 - accuracy: 0.4659
Epoch 9/10
6/6 [==============================] - 0s 25ms/step - loss: 0.6935 - accuracy: 0.4428
Epoch 10/10
6/6 [==============================] - 0s 23ms/step - loss: 0.6948 - accuracy: 0.4969
loaded 300 to

In [ ]:
# gera um arquivo com os pesos do modelo

#test_model.save('saved_test_model')

INFO:tensorflow:Assets written to: saved_test_model/assets


In [ ]:
# usar se estiver usando google colab, isso gera um zip e faz o download do modelo

#!zip -r saved_test_model.zip saved_test_model
#files.download('saved_test_model.zip') 

  adding: saved_test_model/ (stored 0%)
  adding: saved_test_model/variables/ (stored 0%)
  adding: saved_test_model/variables/variables.data-00000-of-00001 (deflated 27%)
  adding: saved_test_model/variables/variables.index (deflated 66%)
  adding: saved_test_model/saved_model.pb (deflated 90%)
  adding: saved_test_model/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# usar se estiver usando google colab, isso descompacta um arquivo no diretório do notebook

#!unzip saved_test_model.zip

Archive:  saved_test_model.zip
   creating: saved_test_model/
   creating: saved_test_model/variables/
  inflating: saved_test_model/variables/variables.data-00000-of-00001  
  inflating: saved_test_model/variables/variables.index  
  inflating: saved_test_model/saved_model.pb  
   creating: saved_test_model/assets/


In [12]:
# carrega um modelo salvo anteriormente

test = load_model('saved_test_model')

In [15]:
# carrega as 1000 primeiras imagens usadas para testar

X_test_img = [crop_center(imread(file), crop_size, crop_size) for file in X_test[:1000]]
print(f'loaded test images')
Y_test_img = Y_test[:1000]
X_test_img = np.stack(X_test_img, axis=0)
X_test_img = X_test_img.reshape((X_test_img.shape + (1,)))
Y_test_img = np.stack(Y_test_img, axis=0).reshape((len(Y_test_img), 1))

loaded test images


In [17]:
# valida o modelo

preds = test.evaluate(X_test_img, Y_test_img, batch_size=32, verbose=1, sample_weight=None)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

32/32 [==============================] - 32s 10ms/step - loss: 0.0494 - accuracy: 0.9830

Loss = 0.04938514530658722
Test Accuracy = 0.9829999804496765
